# Inference of DeepSpot2Cell on H&E Images

In [ ]:
import os
import json
import torch

from deepspot.deepspot2cell import DeepSpot2Cell
from deepspot.utils import get_morphology_model_and_preprocess, predict_spot2cell_from_image_paths


cell_image_path = "path/to/cell_image.png"
spot_image_path = "path/to/spot_image.png"
neighbor_images_paths = ["path/to/neighbor_image1.png", "path/to/neighbor_image2.png", "path/to/neighbor_image3.png", "path/to/neighbor_image4.png"]
model_checkpoint = "results/final_model.pt"
gene_names_path = "path/to/gene_names.json"  # Optional
morphology_model_path = "path/to/phikonv2/model"
output_size = 1000  # Number of genes in the dataset

device = "cuda" if torch.cuda.is_available() else "cpu"

# Load gene names (optional)
gene_names = None
if gene_names_path and os.path.exists(gene_names_path):
    with open(gene_names_path, 'r') as f:
        gene_names = json.load(f)

# Load morphology model
morphology_model, preprocess, input_size = get_morphology_model_and_preprocess("phikonv2", device, model_path=morphology_model_path)
morphology_model.to(device)
morphology_model.eval()

# Load DeepSpot2Cell model
deepspot_model = DeepSpot2Cell(input_size=input_size, output_size=output_size)
checkpoint = torch.load(model_checkpoint, map_location=device)
deepspot_model.load_state_dict(checkpoint)
deepspot_model.to(device)
deepspot_model.eval()
print(f"Models loaded successfully. Input size: {input_size}, Output size: {output_size}")

predictions = predict_spot2cell_from_image_paths(
    cell_image_path=cell_image_path,
    spot_image_path=spot_image_path,
    neighbor_image_paths=neighbor_images_paths,
    deepspot2cell_model=deepspot_model,
    morphology_model=morphology_model,
    preprocess=preprocess,
    device=device,
    gene_names=gene_names
)

In [ ]:
print("\nTop expressed genes:")
for gene, value in sorted(predictions.items(), key=lambda x: x[1], reverse=True)[:20]:
    print(f"{gene}: {value:.4f}")